# Import the Dataset and the Library needed

In [2]:
from google.colab import files
uploaded = files.upload()

Saving IBM-data.csv to IBM-data (1).csv


In [0]:
import pandas as pd
import io
data = pd.read_csv(io.BytesIO(uploaded['IBM-data.csv']))

In [5]:
# sneak peek of the data
data.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,2,Female,94,3,2,Sales Executive,4,Single,5993,19479,8,Y,Yes,11,3,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,3,Male,61,2,2,Research Scientist,2,Married,5130,24907,1,Y,No,23,4,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,4,Male,92,2,1,Laboratory Technician,3,Single,2090,2396,6,Y,Yes,15,3,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,4,Female,56,3,1,Research Scientist,3,Married,2909,23159,1,Y,Yes,11,3,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,1,Male,40,3,1,Laboratory Technician,2,Married,3468,16632,9,Y,No,12,3,4,80,1,6,3,3,2,2,2,2


In [0]:
# import library
import numpy as np
import tensorflow as tf
from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

# Data Preparation for Modeling

## Target

In [0]:
# convert the target into binary (numeric) values
data['Attrition'] = data['Attrition'].map({'Yes': 1, 'No': 0})

## Other Features

In [0]:
feature_columns = []

# numeric cols
for header in ['DailyRate', 'JobSatisfaction', 'WorkLifeBalance']:
  feature_columns.append(feature_column.numeric_column(header))

# bucketized cols
age_buckets = feature_column.bucketized_column(feature_column.numeric_column('Age'), boundaries=[18, 25, 30, 35, 40, 45, 50, 55, 60])
feature_columns.append(age_buckets)

# indicator cols
gender = feature_column.categorical_column_with_vocabulary_list(
      'Gender', ['Male', 'Female'])
gender_one_hot = feature_column.indicator_column(gender)
feature_columns.append(gender_one_hot)

## Train-Test Split

In [9]:
train, test = train_test_split(data, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

940 train examples
236 validation examples
294 test examples


## Create input pipeline and feature layer

In [0]:
def df_to_dataset(data, shuffle=True, batch_size=32):
  data = data.copy()
  labels = data.pop('Attrition')
  ds = tf.data.Dataset.from_tensor_slices((dict(data), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(data))
  ds = ds.batch(batch_size)
  return ds

In [0]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [0]:
batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

# Model

In [14]:
model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(128, activation='relu'),
  layers.Dense(128, activation='relu'),
  layers.Dense(1)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(train_ds,
          validation_data=val_ds,
          epochs=5)

Epoch 1/5
30/30 [==============================] - 0s 9ms/step - loss: 9.2936 - accuracy: 0.7415 - val_loss: 6.5912 - val_accuracy: 0.8305
Epoch 2/5
30/30 [==============================] - 0s 4ms/step - loss: 2.7968 - accuracy: 0.7574 - val_loss: 3.4207 - val_accuracy: 0.1653
Epoch 3/5
30/30 [==============================] - 0s 4ms/step - loss: 1.7981 - accuracy: 0.7372 - val_loss: 1.9230 - val_accuracy: 0.8305
Epoch 4/5
30/30 [==============================] - 0s 4ms/step - loss: 1.0196 - accuracy: 0.7691 - val_loss: 0.5221 - val_accuracy: 0.8305
Epoch 5/5
30/30 [==============================] - 0s 4ms/step - loss: 0.7598 - accuracy: 0.7840 - val_loss: 0.8644 - val_accuracy: 0.8305


In [15]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

10/10 [==============================] - 0s 2ms/step - loss: 0.7328 - accuracy: 0.8571
Accuracy 0.8571428656578064
